# Triển khai Hệ thống Chuyên gia về Động vật

Một ví dụ từ [AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners).

Trong ví dụ này, chúng ta sẽ triển khai một hệ thống dựa trên kiến thức đơn giản để xác định một loài động vật dựa trên một số đặc điểm vật lý. Hệ thống có thể được biểu diễn bằng cây AND-OR sau đây (đây chỉ là một phần của toàn bộ cây, chúng ta có thể dễ dàng thêm một số quy tắc khác):

![](../../../../translated_images/AND-OR-Tree.5592d2c70187f283703c8e9c0d69d6a786eb370f4ace67f9a7aae5ada3d260b0.vi.png)


## Hệ thống chuyên gia của chúng ta với suy luận ngược

Hãy thử định nghĩa một ngôn ngữ đơn giản để biểu diễn tri thức dựa trên các quy tắc sản xuất. Chúng ta sẽ sử dụng các lớp Python làm từ khóa để định nghĩa các quy tắc. Sẽ có 3 loại lớp chính:
* `Ask` đại diện cho một câu hỏi cần được hỏi người dùng. Nó chứa tập hợp các câu trả lời có thể.
* `If` đại diện cho một quy tắc, và nó chỉ là một cách viết ngắn gọn để lưu trữ nội dung của quy tắc.
* `AND`/`OR` là các lớp để biểu diễn các nhánh AND/OR của cây. Chúng chỉ lưu trữ danh sách các đối số bên trong. Để đơn giản hóa mã, tất cả chức năng được định nghĩa trong lớp cha `Content`.


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

Trong hệ thống của chúng tôi, bộ nhớ làm việc sẽ chứa danh sách các **sự kiện** dưới dạng **cặp thuộc tính-giá trị**. Cơ sở tri thức có thể được định nghĩa như một từ điển lớn, ánh xạ các hành động (các sự kiện mới cần được chèn vào bộ nhớ làm việc) với các điều kiện, được biểu diễn dưới dạng biểu thức AND-OR. Ngoài ra, một số sự kiện có thể được `Hỏi`.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

Để thực hiện suy luận ngược, chúng ta sẽ định nghĩa lớp `Knowledgebase`. Lớp này sẽ bao gồm:
* `memory` hoạt động - một từ điển ánh xạ các thuộc tính với giá trị của chúng
* Các `rules` của Knowledgebase theo định dạng đã được định nghĩa ở trên

Hai phương thức chính là:
* `get` để lấy giá trị của một thuộc tính, thực hiện suy luận nếu cần thiết. Ví dụ, `get('color')` sẽ lấy giá trị của ô màu (nó sẽ hỏi nếu cần, và lưu giá trị để sử dụng sau trong bộ nhớ hoạt động). Nếu chúng ta hỏi `get('color:blue')`, nó sẽ hỏi về màu sắc, và sau đó trả về giá trị `y`/`n` tùy thuộc vào màu sắc.
* `eval` thực hiện suy luận thực tế, tức là duyệt cây AND/OR, đánh giá các mục tiêu con, v.v.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

Bây giờ hãy định nghĩa cơ sở kiến thức về động vật của chúng ta và thực hiện tư vấn. Lưu ý rằng lời gọi này sẽ đặt câu hỏi cho bạn. Bạn có thể trả lời bằng cách gõ `y`/`n` cho các câu hỏi có/không, hoặc bằng cách chỉ định số (0..N) cho các câu hỏi có nhiều lựa chọn dài hơn.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## Sử dụng PyKnow để Suy Diễn Tiến

Trong ví dụ tiếp theo, chúng ta sẽ thử triển khai suy diễn tiến bằng cách sử dụng một trong những thư viện dành cho biểu diễn tri thức, [PyKnow](https://github.com/buguroo/pyknow/). **PyKnow** là một thư viện để tạo ra các hệ thống suy diễn tiến trong Python, được thiết kế tương tự như hệ thống cổ điển cũ [CLIPS](http://www.clipsrules.net/index.html).

Chúng ta cũng có thể tự triển khai suy diễn tiến mà không gặp nhiều khó khăn, nhưng các triển khai đơn giản thường không hiệu quả. Để khớp quy tắc hiệu quả hơn, một thuật toán đặc biệt [Rete](https://en.wikipedia.org/wiki/Rete_algorithm) được sử dụng.


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/buguroo/pyknow/

  Cloning https://github.com/buguroo/pyknow/ to /tmp/pip-req-build-3cqeulyl
  Running command git clone --filter=blob:none --quiet https://github.com/buguroo/pyknow/ /tmp/pip-req-build-3cqeulyl
  Resolved https://github.com/buguroo/pyknow/ to commit 48818336f2e9a126f1964f2d8dc22d37ff800fe8
  Preparing metadata (setup.py) ... done
  Using cached frozendict-1.2.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... done
  Using cached schema-0.6.7-py2.py3-none-any.whl (14 kB)
  Created wheel for pyknow: filename=pyknow-1.7.0-py3-none-any.whl size=34228 sha256=b7de5b09292c4007667c72f69b98d5a1b5f7324ff15f9dd8e077c3d5f7aade42
  Stored in directory: /tmp/pip-ephem-wheel-cache-k7jpave7/wheels/81/1a/d3/f6c15dbe1955598a37755215f2a10449e7418500d7bd4b9508
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3148 sha256=2863d55c240d2409cddf05ccfe600591f8478681549fc97555c47c90dc6bb160
  Stored in directory: /home/rg/.cache/pip/wheels/49/ac/f8/cb8120244e710bdb479c86198b03c7b08c

In [13]:
from pyknow import *
#import pyknow

Chúng ta sẽ định nghĩa hệ thống của mình như một lớp kế thừa từ `KnowledgeEngine`. Mỗi quy tắc được định nghĩa bởi một hàm riêng biệt với chú thích `@Rule`, xác định khi nào quy tắc nên được kích hoạt. Bên trong quy tắc, chúng ta có thể thêm các sự kiện mới bằng cách sử dụng hàm `declare`, và việc thêm các sự kiện đó sẽ dẫn đến việc một số quy tắc khác được gọi bởi bộ suy diễn tiến.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

Khi chúng ta đã định nghĩa một cơ sở tri thức, chúng ta sẽ điền vào bộ nhớ làm việc của mình với một số sự kiện ban đầu, sau đó gọi phương thức `run()` để thực hiện suy luận. Bạn có thể thấy kết quả là các sự kiện suy luận mới được thêm vào bộ nhớ làm việc, bao gồm cả sự kiện cuối cùng về con vật (nếu chúng ta thiết lập tất cả các sự kiện ban đầu một cách chính xác).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])


---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn thông tin chính thức. Đối với các thông tin quan trọng, khuyến nghị sử dụng dịch vụ dịch thuật chuyên nghiệp từ con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
